In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive') # 授权 Google Drive
sys.path.append('/content/drive/MyDrive/Colab Notebooks/utils/clip') # 此为你的自定义模块的存储路径

Mounted at /content/drive


In [2]:
!pip install faiss-cpu

     |████████████████████████████████| 8.6 MB 28.7 MB/s 


In [3]:
!pip install ftfy

     |████████████████████████████████| 53 kB 2.1 MB/s 


In [4]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import clip
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import re
from simple_tokenizer import SimpleTokenizer
import faiss
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
# Load CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("/content/drive/MyDrive/Colab Notebooks/utils/clip/ViT-B-32.pt", device=device, jit=False)

# Get embedding size
embed_dim = model.text_projection.shape[1]
embed_dim

512

In [7]:
from PIL import Image
import requests
 
# 加载图片
image = Image.open("/content/drive/MyDrive/shopee-product-matching/train_images/00a5b7d85f39222ad75b0d69886a6b58.jpg")
print("Image to be processed")
display(image)
 
# 预处理图像
image = preprocess(image).unsqueeze(0).to(device)
print("\n\nTensor shape:")
print(image.shape)

OSError: ignored

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(image)
print(image_features)

In [ ]:
text_snippets = ["The Body Shop Chinese Ginseng & Rice Clarifying Mask 15Ml", 
                 "BSB gantungan hp strap phone tali leher motif sport all type hp",
                 "Crystallure Supreme Revitalizing Oil Serum 30ml"]
 
# 预处理文本
text = clip.tokenize(text_snippets).to(device)
print(text.shape)
with torch.no_grad():
    text_features = model.encode_text(text)
print(text_features.shape)

In [ ]:
with torch.no_grad():
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()
 
print("Label probs:", probs)